In [3]:
#We first import numpy as a math package, networkx to work with graphs and matplotlib to create plots.
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
#This function pseudorandomly generates a graph, with standard randomseed 1(can be changed by giving random seed as third argument).
#Input int n (number of vertices), float prob(probability of there being a connection), randomseed=1 (value for our seudorandomgenerator to consistently generate the same random graph)
#Returns int len(vertices), which is the total amount of vertices in the graph, and list edgeset containing all edges of the random graph.
def genRandomGraph(n,prob,randomseed = 1):
    #Generate the vertex set based on the given number of vertices
    vertices = []
    for i in range(0,n):
            vertices.append(i)
    #Create a pseudorandom edgeset corresponding to the vertexset based on given probability and number of vertices.
    edgeset = []
    np.random.seed(randomseed)
    nsquared = n*n
    ncubed = n*n*n
    #Create enough zeroes and ones corresponding to all possible edges in the list random_connections
    random_connections = np.random.binomial(1,prob,size = ncubed)
    #Add all edges corresponding to a 1 in random connections to the edgeset.
    for i in range(0,len(vertices)):
        for j in range(i+1, len(vertices)):
            if (int(random_connections[i*nsquared + j]) == 1):
                edgeset.append((i, j))
    
    return [len(vertices), edgeset]

In [5]:
#This function generates a graph based on gives vertices and edges and tests if this graph is connected.
#Input int numberOfVertices, list edges
#Returns boolean connectedBoolean
def isConnected(numberOfVertices, edges):
    #create empty Graph G with networkx
    G = nx.Graph()
    #Add vertices to Graph G based on input numberOfVertices
    for vertex in range(0,numberOfVertices):
        G.add_node(vertex)
    #Add edges to graph G based on input edges
    for edge in edges:
        G.add_edge(edge[0],edge[1])
    #check if graph G is connected, store result as Boolean and return it.
    connectedBoolean = nx.is_connected(G)
    return connectedBoolean

In [6]:
#This function creates a random connected graph based on the number of vertices and the average probability of edges being in the graph.
def generateConnectedGraph(n, prob):
    random_seed = 0
    random_graph = genRandomGraph(n, prob, random_seed)
    connectedBoolean = False
    while connectedBoolean == False:
        random_seed += 1
        random_graph = genRandomGraph(n,prob,random_seed)
        connectedBoolean = isConnected(random_graph[0], random_graph[1])
    print("Random seed = " + str(random_seed))
    return (random_graph)

In [7]:
#This function determines the vertices of grade 0 in a graph with list edgeset and n vertices and returns the vertices with grade 0 as a list.
def dertermineFreeVertices(edges, n):
    index = list(range(0,n))
    numbersWithEdges = []
    for edge in edges:
        numbersWithEdges.append(edge[0])
        numbersWithEdges.append(edge[1])
    numbersWithEdges = set(numbersWithEdges)
    verticesWithoutEdges = list(set(index)-set(numbersWithEdges))
    return verticesWithoutEdges

In [8]:
#This function calculates the Ideal and Groebnerbasis over Q and does not define a NumberField.
def calculateOverQ(graphInfo, k, ordering, verticesWithoutEdges):
    numberOfVertices = graphInfo[0]
    R = PolynomialRing(QQ,numberOfVertices, names='x', order=ordering)
    x = R.gens()
    edges = graphInfo[1]
    index = list(range(0,n))
    functions = []
    
    if is_even(k):
        for i in index:
            functions.append(prod([x[i]^2-j^2 for j in range(1,k/2 + 1)]))
        for edge in edges:
            functions.append(R(factor((prod([x[edge[0]]^2-j^2 for j in range(1,k/2+1)]) - prod([x[edge[1]]^2-j^2 for j in range(1,k/2+1)]))/(x[edge[0]]-x[edge[1]]))))
    else:
        for i in index:
            functions.append(x[i]*prod([x[i]^2-j^2 for j in range(1,(k-1)/2 + 1)]))
        for edge in edges:
            functions.append(R(factor((x[edge[0]]*prod([x[edge[0]]^2-j^2 for j in range(1,(k-1)/2+1)]) - x[edge[1]]*prod([x[edge[1]]^2-j^2 for j in range(1,(k-1)/2+1)]))/(x[edge[0]]-x[edge[1]]))))
        
    substitutionDictionairy = {}
    
    for vertex in verticesWithoutEdges:
        substitutionDictionairy[x[vertex]] = 1
        
    firstConnectedVertex = edges[0][0]
    secondConnectedVertex = edges[0][1]
    
    substitutionDictionairy[x[firstConnectedVertex]] = 1
    substitutionDictionairy[x[secondConnectedVertex]] = -1
    
    
    functions = [f.subs(substitutionDictionairy) for f in functions]    
    for vertex in verticesWithoutEdges:
        functions.append(x[vertex]-1)
    
    functions.append(x[firstConnectedVertex] - 1)
    functions.append(x[secondConnectedVertex] + 1)
    
    
    I = Ideal(functions)
    G = I.groebner_basis()
    
    return (I,G)

In [9]:
def createPolynomialRing(numberOfVertices, k, ordering):
    F = createNumberField(k)
    R = PolynomialRing(F,numberOfVertices, names='x', order=ordering)
     
    return R

In [10]:
#This function calculates the Ideal and Groebnerbasis with a PolynomialRing based on a NumberField
def calculateOverCyclictomical(graphInfo, k, ordering, verticesWithoutEdges):
    numberOfVertices = graphInfo[0]
    R.<w> = PolynomialRing(QQ)
    b = [0,w-1, w+1, w^(2)+w+1, w^(2)+1, w^(4)+w^(3)+w^(2)+w+1, w^(2)-w+1, w^(6)+w^(5)+w^(4)+w^(3)+w^(2)+w+1, w^(4)+1, w^(6)+w^(3)+1]
    cyclitomicPolynomial = b[k]
    F.<w> = NumberField(cyclitomicPolynomial)
    R = PolynomialRing(F,numberOfVertices, names='x', order=ordering)
    
    
    edges = graphInfo[1]
    
    
    x = R.gens()
    index = list(range(0,n))
    functions = []
    
    for i in index:
        functions.append(x[i]^k - 1)
    
    for edge in edges:
        functions.append(sum([x[edge[0]]^(k-1-m) * x[edge[1]]^m for m in range(0,k)]))
        
    substitutionDictionairy = {}
    
    for vertex in verticesWithoutEdges:
        substitutionDictionairy[x[vertex]] = 1
        
    firstConnectedVertex = edges[0][0]
    secondConnectedVertex = edges[0][1]
    
    substitutionDictionairy[x[firstConnectedVertex]] = 1
    substitutionDictionairy[x[secondConnectedVertex]] = w
    
    
    functions = [f.subs(substitutionDictionairy) for f in functions]    
    for vertex in verticesWithoutEdges:
        functions.append(x[vertex]-1)
    
    functions.append(x[firstConnectedVertex] - 1)
    functions.append(x[secondConnectedVertex] - w)
    
    
    I = Ideal(functions)
    G = I.groebner_basis()
    
    return (I,G)

In [11]:
#Generate a random connected graph G, to be colored later using different coloring methods.
n=22
p=0.2
ordering='degrevlex'
graphInfo = generateConnectedGraph(n,p)
verticesWithoutEdges = dertermineFreeVertices(graphInfo[1], n)
print(graphInfo)

Random seed = 1
[22, [(0, 13), (0, 20), (0, 21), (1, 7), (1, 11), (1, 12), (1, 13), (1, 14), (2, 3), (2, 5), (2, 14), (2, 20), (2, 21), (3, 4), (3, 9), (3, 10), (4, 13), (4, 16), (5, 10), (5, 17), (5, 19), (6, 9), (6, 13), (6, 21), (7, 8), (7, 12), (7, 14), (8, 16), (8, 19), (8, 21), (9, 18), (10, 17), (10, 18), (11, 14), (11, 19), (11, 21), (12, 19), (13, 17), (13, 18), (14, 15), (14, 16), (14, 18), (14, 20), (15, 16), (15, 18), (15, 19), (16, 18)]]


In [12]:
#Calculate coloring of graph G over Q without defining NumberField
k=1
groebnerBasis = [1]
while groebnerBasis == [1]:
    k += 1
    print(k)
    (I, groebnerBasis) = calculateOverQ(graphInfo, k, ordering, verticesWithoutEdges)
    
print('The chromatic number of the graph is: ', k)
#number_of_solutions = I.radical().vector_space_dimension()*k^(len(verticesWithoutEdges)+1)*(k-1)
#print('The number of colorings using at most', k, 'colors is: ', number_of_solutions)




2


In [0]:
#Calculate coloring of G with PolynomialRing based on NumberField
k=1
ordering = 'degrevlex'
groebnerBasis = [1]
while groebnerBasis == [1]:
    k += 1
    print(k)
    (I, groebnerBasis) = calculateOverCyclictomical(graphInfo, k, ordering, verticesWithoutEdges)
    
print('The chromatic number of the graph is: ', k)
#number_of_solutions = I.radical().vector_space_dimension()*k^(len(verticesWithoutEdges)+1)*(k-1)
#print('The number of colorings using at most', k, 'colors is: ', number_of_solutions)
